In [1]:
import pandas as pd
import os
import torch
from model.database_util import *

In [2]:
import torch
from model.util import Normalizer

# cost_norm = Normalizer(1, 100)
# cost_norm = Normalizer(-3.61192, 12.290855)
#cost_norm = Normalizer(5, 2611)
cost_norm = Normalizer(8.26, 11.12)

## Define Model

In [3]:
class Args:
    # bs = 1024
    # SQ: smaller batch size
    bs = 1
    #lr = 0.001
    lr = 0.001
    # epochs = 200
    epochs = 50
    clip_size = 50
    embed_size = 64
    pred_hid = 128
    ffn_dim = 128
    head_size = 12
    n_layers = 8
    dropout = 0.1
    sch_decay = 0.6
    # device = 'cuda:0'
    device = 'cpu'
    newpath = 'job_queries_training'
    to_predict = 'cost'
args = Args()

import os
if not os.path.exists(args.newpath):
    os.makedirs(args.newpath)

In [4]:
from model.model import QueryFormer

model = QueryFormer(emb_size = args.embed_size ,ffn_dim = args.ffn_dim, head_size = args.head_size, \
                 dropout = args.dropout, n_layers = args.n_layers, \
                 use_sample = False, use_hist = False, \
                 pred_hid = args.pred_hid
                )

In [5]:
from model.dataset import PlanTreeDataset

# Loading tensors

In [6]:
# Load the saved tensor collection
loaded_tensors = torch.load("./job_queries/tensors/query_1_2024-12-03-20.43.44.200877.pt")

# Access each tensor
x_loaded = loaded_tensors["x"]
rel_pos_loaded = loaded_tensors["rel_pos"]
attn_bias_loaded = loaded_tensors["attn_bias"]
heights_loaded = loaded_tensors["heights"]
cost_labels_loaded = loaded_tensors["cost_labels"]
raw_costs_loaded = loaded_tensors["raw_costs"]

# Print the shapes to verify
print(f"x shape: {x_loaded.shape}")
print(f"rel_pos shape: {rel_pos_loaded.shape}")
print(f"attn_bias shape: {attn_bias_loaded.shape}")
print(f"heights shape: {heights_loaded.shape}")
print(f"cost_labels shape: {cost_labels_loaded.shape}")
print(f"raw_costs shape: {raw_costs_loaded.shape}")

x shape: torch.Size([1, 30, 1161])
rel_pos shape: torch.Size([1, 30, 30])
attn_bias shape: torch.Size([1, 31, 31])
heights shape: torch.Size([1, 30])
cost_labels shape: torch.Size([1])
raw_costs shape: torch.Size([1])


In [7]:
x_loaded

tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [2., 0., 0.,  ..., 1., 1., 1.],
         [3., 1., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]])

In [8]:
rel_pos_loaded

tensor([[[ 1,  2,  3,  4,  4,  5,  5,  6,  6,  7,  8,  8,  9, 10, 10, 11, 11,
           9,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61,  1,  2,  3,  3,  4,  4,  5,  5,  6,  7,  7,  8,  9,  9, 10, 10,
           8,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61,  1,  2,  2,  3,  3,  4,  4,  5,  6,  6,  7,  8,  8,  9,  9,
           7,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61,  1, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
          61, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61,  1,  2,  2,  3,  3,  4,  5,  5,  6,  7,  7,  8,  8,
           6,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61, 61,  1, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61,
          61, 61,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61, 61, 61, 61, 61, 61,  1,  2,  2,  3,  4,  4,  5,  6,  6,  7,  7,
           5,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [61,

In [9]:
attn_bias_loaded

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
       

In [10]:
heights_loaded

tensor([[12, 11, 10,  2,  9,  2,  8,  2,  7,  6,  2,  2,  5,  4,  2,  2,  3,  2,
          2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [11]:
raw_costs_loaded

tensor([66844.], dtype=torch.float64)

In [12]:
cost_labels_loaded

tensor([0.9965], dtype=torch.float64)

In [13]:
# Print the shapes to verify
print(f"x shape: {x_loaded.shape}")
print(f"rel_pos shape: {rel_pos_loaded.shape}")
print(f"attn_bias shape: {attn_bias_loaded.shape}")
print(f"heights shape: {heights_loaded.shape}")
print(f"cost_labels shape: {cost_labels_loaded.shape}")
print(f"raw_costs shape: {raw_costs_loaded.shape}")

x shape: torch.Size([1, 30, 1161])
rel_pos shape: torch.Size([1, 30, 30])
attn_bias shape: torch.Size([1, 31, 31])
heights shape: torch.Size([1, 30])
cost_labels shape: torch.Size([1])
raw_costs shape: torch.Size([1])


## TEST - Loading 2 tensors

In [14]:
# Load the saved tensor collection
loaded_tensors1 = torch.load("./job_queries/tensors/query_1_2024-12-03-20.43.44.200877.pt")

# Access each tensor
x_loaded1 = loaded_tensors["x"]
rel_pos_loaded1 = loaded_tensors["rel_pos"]
attn_bias_loaded1 = loaded_tensors["attn_bias"]
heights_loaded1 = loaded_tensors["heights"]
cost_labels_loaded1 = loaded_tensors["cost_labels"]
raw_costs_loaded1 = loaded_tensors["raw_costs"]

# Load the saved tensor collection
loaded_tensors2 = torch.load("./job_queries/tensors/query_2_2024-12-03-20.43.45.954590.pt")

# Access each tensor
x_loaded2 = loaded_tensors["x"]
rel_pos_loaded2 = loaded_tensors["rel_pos"]
attn_bias_loaded2 = loaded_tensors["attn_bias"]
heights_loaded2 = loaded_tensors["heights"]
cost_labels_loaded2 = loaded_tensors["cost_labels"]
raw_costs_loaded2 = loaded_tensors["raw_costs"]

In [15]:
x_list = [x_loaded1, x_loaded2]
rel_pos_list = [rel_pos_loaded1, rel_pos_loaded2]
attn_bias_list = [attn_bias_loaded1, attn_bias_loaded2]
heights_list = [heights_loaded1, heights_loaded2]
cost_labels_list = [cost_labels_loaded1, cost_labels_loaded2]
raw_costs_list = [raw_costs_loaded1, raw_costs_loaded2]

In [16]:
# Initialize dataset
dataset = PlanTreeDataset(
    2,
    x_list,
    attn_bias_list,
    rel_pos_list,
    heights_list,
    cost_labels_list,
    raw_costs_list
)

print("Dataset length:", len(dataset))
print("First sample:", dataset[0])

Dataset length: 2
First sample: ({'x': tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [2., 0., 0.,  ..., 1., 1., 1.],
         [3., 1., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), 'attn_bias': tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0

In [17]:
num_copies = 2

# Replicate data
replicated_x = [x_loaded.clone() for _ in range(num_copies)]
replicated_attn_bias = [attn_bias_loaded.clone() for _ in range(num_copies)]
replicated_rel_pos = [rel_pos_loaded.clone() for _ in range(num_copies)]
replicated_heights = [heights_loaded.clone() for _ in range(num_copies)]
replicated_cost_labels = [cost_labels_loaded.clone() for _ in range(num_copies)]  # Tensor replicated
replicated_raw_costs = [raw_costs_loaded.clone() for _ in range(num_copies)]  # List replicated


In [18]:

# Initialize dataset
dataset = PlanTreeDataset(
    num_copies,
    replicated_x,
    replicated_attn_bias,
    replicated_rel_pos,
    replicated_heights,
    replicated_cost_labels,
    replicated_raw_costs
)

print("Dataset length:", len(dataset))
print("First sample:", dataset[0])

Dataset length: 2
First sample: ({'x': tensor([[[1., 0., 0.,  ..., 1., 1., 1.],
         [2., 0., 0.,  ..., 1., 1., 1.],
         [3., 1., 0.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), 'attn_bias': tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0., -inf, -inf, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf,
          -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
         [0

In [19]:
type(replicated_x)

list

In [20]:
assert len(replicated_x) == num_copies
assert len(replicated_attn_bias) == num_copies
assert len(replicated_rel_pos) == num_copies
assert len(replicated_heights) == num_copies
assert len(replicated_cost_labels) == num_copies
assert len(replicated_raw_costs) == num_copies

In [21]:
# Initialize the PlanTreeDataset with optional costs
#dataset = PlanTreeDataset(1, [x], [attn_bias], [rel_pos], [heights], cost_labels, raw_costs)


In [22]:
# Get the length of the dataset
print(f"Dataset size: {len(dataset)}")

# Access a single sample
sample, label = dataset[0]

# Print the sample contents
print("Sample contents:")
print("Feature Matrix (x):", sample['x'].shape)
print("Attention Bias (attn_bias):", sample['attn_bias'].shape)
print("Relative Positions (rel_pos):", sample['rel_pos'].shape)
print("Heights (heights):", sample['heights'].shape)
print("Label:", label)

Dataset size: 2
Sample contents:
Feature Matrix (x): torch.Size([1, 30, 1161])
Attention Bias (attn_bias): torch.Size([1, 31, 31])
Relative Positions (rel_pos): torch.Size([1, 30, 30])
Heights (heights): torch.Size([1, 30])
Label: (tensor([0.9965], dtype=torch.float64), tensor([66844.], dtype=torch.float64))


In [23]:
# Example numpy label
import numpy as np
import torch.nn as nn
import importlib

from model import trainer
importlib.reload(trainer)
from  model.trainer import train_single, train


crit = nn.MSELoss()

# Train the model with the numpy label
# trained_model = train_single(model, dataset, dataset, crit, cost_norm, args)
model, best_model_path, train_embeddings, val_embeddings = train(model, dataset, dataset, crit, cost_norm, args)


pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])


/home/db2inst1/learnedwmp/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
Epoch: 0  Avg Loss: 0.17684863531030715, Time: 2.732783317565918
Median: 3.31295773519345
Mean: 3.31295773519345
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Labe

/home/db2inst1/learnedwmp/templates/job-templatizing-queryformer-modified/model/trainer.py:42: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(np.log(ps), np.log(ls))


pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])


...

pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])


/home/db2inst1/learnedwmp/.venv/lib/python3.12/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])


/home/db2inst1/learnedwmp/templates/job-templatizing-queryformer-modified/model/trainer.py:42: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, _ = pearsonr(np.log(ps), np.log(ls))


pred_output shape: torch.Size([1])
Cost Predictions Shape: torch.Size([])
Batch Cost Label Shape: torch.Size([1])
pred_output shape: torch.Size([1])
pred_output shape: torch.Size([1])


In [24]:
print(f"Number of training embeddings (best epoch): {len(train_embeddings)}")
print(f"First training embedding shape: {train_embeddings[0].shape}")
print(f"Number of validation embeddings (best epoch): {len(val_embeddings)}")
print(f"First validation embedding shape: {val_embeddings[0].shape}")


Number of training embeddings (best epoch): 2
First training embedding shape: (1417,)
Number of validation embeddings (best epoch): 2
First validation embedding shape: (1417,)


In [25]:
print(f"First validation embedding shape: {val_embeddings[0]}")

First validation embedding shape: [ 2.851161   20.531675    8.309177   ... 23.04402     4.2916102
 -0.70323336]


In [26]:
type(val_embeddings[0])

numpy.ndarray

In [27]:
print(f"First validation embedding shape: {val_embeddings[1]}")

First validation embedding shape: [ 2.851161   20.531675    8.309177   ... 23.04402     4.2916102
 -0.70323336]


In [28]:
len(val_embeddings[0])

1417

In [29]:
import numpy as np

# Disable truncation
np.set_printoptions(threshold=np.inf, linewidth=1000)

for i, emb in enumerate(val_embeddings):
    print(f"Full Embedding {i}:\n{emb}")


Full Embedding 0:
[ 2.85116100e+00  2.05316753e+01  8.30917740e+00  2.77599354e+01  4.93734026e+00  5.56273651e+00  1.63346729e+01  2.36758995e+01  1.85432415e+01 -1.16897087e+01 -3.79956460e+00 -6.44580650e+00  5.27722692e+00 -5.79063606e+00  2.32282734e+01  1.37613554e+01 -2.06316757e+00  1.43070784e+01  7.50589943e+00  4.52252531e+00  1.11168356e+01 -2.93024063e+00 -1.60223408e+01  9.50275421e+00  8.65117550e-01 -9.10232830e+00 -1.42041359e+01 -1.67259426e+01 -2.07097797e+01  7.96444845e+00  1.29832888e+00  8.79665184e+00  2.46938286e+01 -3.13759661e+00 -1.47409973e+01 -3.42307329e+00 -8.52990532e+00 -6.18487024e+00 -2.07017732e+00 -6.73620558e+00  1.18425636e+01  1.78384900e+00  6.30518103e+00 -1.48744545e+01 -1.87404194e+01  1.53625002e+01  2.29943504e+01 -4.10873079e+00  9.99921227e+00 -1.58318691e+01  2.12915611e+01  1.41741533e+01 -1.25254164e+01 -2.31318970e+01  1.03742981e+01  2.48843918e+01  1.68221073e+01 -2.60557690e+01 -1.14073906e+01 -8.00450134e+00 -2.11683826e+01  6.30